# Time-Series Generation using Contrastive Learning

Consider learning a generative model for time-series data.

The sequential setting poses a unique challenge: Not only should the generator capture the conditional dynamics of (stepwise) transitions, but its open-loop rollouts should also preserve the joint distribution of (multi-step) trajectories.

On one hand, autoregressive models
trained by MLE allow learning and computing explicit transition distributions, but suffer from compounding error during rollouts.

On the other hand, adversarial models based on GAN training alleviate such exposure bias, but transitions are implicit and hard to assess.

In this work, we study a generative framework that seeks to combine the strengths of both: Motivated by a moment-matching objective to mitigate
compounding error, we optimize a local (but forward-looking) *transition policy*, where the reinforcement signal is provided by a global (but stepwise-decomposable) *energy model* trained by contrastive estimation. 

At **training**, the two components are learned cooperatively, avoiding the instabilities typical of adversarial objectives. 

At **inference**, the learned policy serves as the generator for iterative sampling, and the learned energy serves as a trajectory-level measure for evaluating sample quality.

By expressly training a policy to imitate sequential behavior of time-series features in a dataset, this approach embodies *“generation by imitation”*. Theoretically, we illustrate the correctness of this formulation and the consistency of the algorithm.

Empirically, we evaluate its ability to generate predictively useful samples from real-world datasets, verifying that it performs at the standard of existing benchmarks.

## 1 Setup

### 1.1 Install libraries

Run the cell below to **install** the necessary libraries.

In [1]:
%pip install wandb
%pip install pytorch-lightning
%pip install plotly
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### 1.2 Import Libraries

Run the cell below to **import** the necessary libraries

In [2]:
import random
import numpy as np
import torch

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2024-02-06 21:00:16.770219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 21:00:16.770251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 21:00:16.771351: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 21:00:16.776939: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU 

In [4]:
import wandb
from pytorch_lightning.loggers.wandb import WandbLogger

Eh eh

In [5]:
import warnings
warnings.filterwarnings("ignore")

### 1.3 Hyper-parameters

The cell below contains *all* the hyper-parameters nedded by this script, for easy tweaking.

In [6]:
from hyperparamets import Config
hparams = Config()

Comment this cell if you don't want to use Weights & Biases to log the process

In [7]:
#!wandb login

### 1.5 Initialization

Initialize the modules needed by running the cells in this section.

#### 1.5.1 reproducibility.

In [8]:
np.random.seed(0)
random.seed(0)

torch.cuda.manual_seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True # Can have performance impact
torch.backends.cudnn.benchmark = False

_ = pl.seed_everything(0)

Global seed set to 0


#### 1.5.2 Device

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}.")

Using device cpu.


#### 1.5.3 Data

Generate the dataset as requested

In [10]:
import os
from hyperparamets import Config
print(os.listdir('.'))
hparams = Config()

['data_generation', '__pycache__', 'utilities.py', 'tests.ipynb', 'modules', 'timegan_model.py', 'training_loop.py', 'hyperparamets.py', 'main.ipynb', 'dataset_handling.py']


In [11]:
from data_generation import iid_sequence_generator, sine_process, wiener_process

In [12]:
if hparams.dataset_name in ['sine', 'wien', 'iid', 'cov']:
  # Generate and store the dataset as requested
  if hparams.dataset_name == 'sine':
    sine_process.save_sine_process(p=hparams.data_dim, N=hparams.num_samples, file_path="../datasets/sine_generated_stream.csv")
  elif hparams.dataset_name == 'wien':
    wiener_process.save_wiener_process(p=hparams.data_dim, N=hparams.num_samples, file_path="../datasets/wien_generated_stream.csv")
  elif hparams.dataset_name == 'iid':
    iid_sequence_generator.save_iid_sequence(p=hparams.data_dim, N=hparams.num_samples, file_path="../datasets/iid_generated_stream.csv")
  elif hparams.dataset_name == 'cov':
    iid_sequence_generator.save_cov_sequence(p=hparams.data_dim, N=hparams.num_samples, file_path="../datasets/iid_generated_stream.csv")
  else:
    raise ValueError
  print(f"The {hparams.dataset_name} dataset has been created successfully.")
elif hparams.dataset_name == 'real':
  pass
else:
  raise ValueError("Dataset not supported.")

Computed 1-th weiner process.
Computed 2-th weiner process.
Computed 3-th weiner process.
Computed 4-th weiner process.
Computed 5-th weiner process.
Computed 6-th weiner process.
The wien dataset has been created successfully.


Train / Test split

In [13]:
from dataset_handling import train_test_split

In [14]:
if hparams.dataset_name in ['sine', 'wien', 'iid', 'cov']:
    dataset_path       = f"../datasets/{hparams.dataset_name}_generated_stream.csv"
    train_dataset_path = f"../datasets/{hparams.dataset_name}_training.csv"
    test_dataset_path  = f"../datasets/{hparams.dataset_name}_testing.csv"

    train_test_split(X=np.loadtxt(dataset_path, delimiter=",", dtype=np.float32),
                    split=hparams.train_test_split,
                    train_file_name=train_dataset_path,
                    test_file_name=test_dataset_path    
                    )
elif hparams.dataset_name == 'real':
    train_dataset_path = hparams.train_file_path
    test_dataset_path  = hparams.test_file_path
else:
  raise ValueError("Dataset not supported.")

100000


## 1.6 Model

This cell loads the TimeGAN model class.

In [15]:
from timegan_model import TimeGAN

## 2 Train

This chapter will train the model according to the hyper-parameters defined above in section [Hyper-parameters](#13-hyper-parameters).

In [16]:
accelerator = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
# Instantiate the model
timegan = TimeGAN(hparams=hparams,
                    train_file_path=train_dataset_path,
                    test_file_path=test_dataset_path
                    )

In [18]:
# Define the logger -> https://www.wandb.com/articles/pytorch-lightning-with-weights-biases.
wandb_logger = WandbLogger(project="TimeGAN PyTorch (2024)", log_model=True)
wandb_logger.experiment.watch(timegan, log='all', log_freq=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 characters long, yours was 1
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: ERROR API key must be 40 chara

In [ ]:
# Define the trainer
early_stop = EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=hparams.early_stop_patience,
    strict=False,
    verbose=False
)
trainer = Trainer(logger=wandb_logger,
                max_epochs=hparams.n_epochs,
                accelerator=accelerator,
                val_check_interval=0.25,
                callbacks=[early_stop]
                )

In [ ]:
# Start the training
trainer.fit(timegan)

In [ ]:
# Log the trained model
trainer.save_checkpoint('timegan.pth')
wandb.save('timegan.pth')

# 3 Validation

## 4 Visualize Results